In [18]:
import numpy as np
import os
import pandas as pd
import netCDF4 as cdf
from datetime import datetime, timedelta
from scipy.interpolate import griddata

In [19]:
year = "2016"
explotion_times_directory = "explotion-times/"
nc_directory = "carra-files/param_130.nc"

## Open and read .nc file variables


In [20]:
nc = cdf.Dataset(nc_directory)
longitudes = nc.variables["longitude"][100:475, 477:]
latitudes = nc.variables["latitude"][100:475, 477:]
raw_times = nc.variables["time"][:]
temperture = nc.variables["t"][..., 100:475, 477:]
# wind_direction = nc.variables['u'][..., 100:500, 450:]
# wind_speed = nc.variables['v'][..., 100:500, 450:]
nc.close()

## Organize data


In [21]:
new_longitudes = np.where(longitudes <= 180, longitudes, longitudes - 360)

reshaped_longitudes = new_longitudes.view().reshape(np.size(longitudes), 1)
reshaped_latitudes = latitudes.view().reshape(np.size(latitudes), 1)
coords_full_set = np.hstack((reshaped_longitudes, reshaped_latitudes))

longitudes_to_interp = np.linspace(25.5, 25, 72).reshape(72, 1)
latitudes_to_interp = np.linspace(66, 69, 72).reshape(72, 1)
coords_to_interp = np.block([longitudes_to_interp, latitudes_to_interp])

In [29]:
arc_files = [
    filename
    for filename in os.listdir(explotion_times_directory)
    if filename[0:4] == year
]


whole_dates = np.array(
    [
        datetime.utcfromtimestamp(posix_timestamp).strftime("%Y-%m-%dT%H:%M")
        for posix_timestamp in raw_times
    ],
    dtype="datetime64[m]",
)

explosion_dates = np.empty(len(arc_files), dtype="datetime64[m]")


for index, file in enumerate(arc_files):
    data = pd.read_csv(
        explotion_times_directory + file, delim_whitespace=True, header=None
    )

    explotion = np.argmax(data.iloc[:, 3])
    date_of_explotion = data.iloc[explotion, 0]
    year_of_explotion = date_of_explotion[0:4]
    day_number_of_explotion = date_of_explotion[5:8]
    hour_of_explotion = date_of_explotion[9:11]
    minutes_of_explotion = date_of_explotion[12:14]

    day_number_of_explotion = day_number_of_explotion.rjust(
        3 + len(day_number_of_explotion), "0"
    )
    template_date = datetime(
        int(year_of_explotion), 1, 1, int(hour_of_explotion), int(minutes_of_explotion)
    )
    resulting_object_date = template_date + timedelta(
        days=int(day_number_of_explotion) - 1
    )
    resulting_string_date = resulting_object_date.strftime("%Y-%m-%dT%H:%M")
    explosion_dates[index] = resulting_string_date


def date_cleaner(whole_dates, explosion_dates):
    def index_finder(datetime_array, explosion_date):
        indices = np.argsort(np.abs(datetime_array - explosion_date))[:2]
        return indices

    desired_indices = []
    with np.nditer([explosion_dates], flags=["buffered"], op_flags=["readonly"]) as it:
        for explosion_date in it:
            desired_indices.extend(index_finder(whole_dates, explosion_date))

    return desired_indices


desired_indices = np.unique(date_cleaner(whole_dates, explosion_dates))
cleaned_dates = whole_dates[desired_indices]


def request_to_interpolate_again():
    user_choice = input("Volver a hacer otra interpolación?\n1. Sí\n2. No\nRespuesta: ")

    if (user_choice.strip().capitalize() in ["Sí", "Si"]) or (user_choice == "1"):
        kind_of_measurement = int(
            input(
                "1. Temperatura\n2. Velocidd del viento\n3. Dirección del viento\nIntroduce el número de la medida a interpolar: "
            )
        )

        while kind_of_measurement in range(1, 4):
            print("\nInterpolación en proceso...\n")
            first_interpolator(kind_of_measurement, cleaned_dates)
            kind_of_measurement = False

    else:
        print("\nEntendido\n")

    """ Aquí posiblemente haya una función que devuelva un archivo csv o txt que almacene los datos de las interpolaciones deseadas. """


def first_interpolator(kind_of_measurement, selected_time, out=None):
    def second_interpolator(sequence_of_interpolants):
        array_of_interpolants = np.array(sequence_of_interpolants)
        print(np.shape(array_of_interpolants))
        # shape(heigths, times, interpolants)
        index = 0
        with np.nditer(
            [array_of_interpolants, out],
            flags=["buffered", "multi_index", "refs_ok"],
            op_flags=[["readonly"], ["writeonly", "allocate", "no_broadcast"]],
        ) as it:
            for measurement_values, interpolants_for_a_time in it:
                interpolant = np.interp(
                    explosion_dates,
                    cleaned_dates.ravel(),
                    [
                        measurement_values[..., index],
                        measurement_values[..., index + 1],
                    ],
                )
                interpolants_for_a_time[...] = interpolant
                index += 1
                result = it.operands[1]
        print(result)

        print("\nInterpolación temporal exitosa\n")
        return request_to_interpolate_again()

    if kind_of_measurement == 1:
        cleaned_temperture = temperture[desired_indices, ...]

        with np.nditer(
            [np.arange(0, 23, 1)]  ####
        ) as it:  # en teoría aquí va un 23, por 23 alturas
            for height in it:
                measurements_at_same_height = (
                    []
                )  # aquí sería un arreglo creado con np.empty
                with np.nditer(
                    [np.arange(0, len(cleaned_dates), 1)]
                ) as it:  # aquí el tamaño del arreglo cleaned dates
                    for time in it:
                        flatten_temperture = cleaned_temperture[
                            time, height, ...
                        ].ravel()
                        temperture_interpolants = griddata(
                            coords_full_set,
                            flatten_temperture,
                            coords_to_interp,
                            method="cubic",
                        )
                        measurements_at_same_height.append(
                            list(temperture_interpolants)
                        )
                        # measurements_at_same_height[index] = list(temperture_interpolants) en caso de usar el arreglo creado con np.empty
                sequence_of_interpolants.append(measurements_at_same_height)

    print("Interpolación espacial exitosa")
    second_interpolator(sequence_of_interpolants)


sequence_of_interpolants = []

kind_of_measurement = int(
    input(
        "1. Temperatura\n2. Velocidad del viento\n3. Dirección del viento\nIntroduce el número de la medida a interpolar: "
    )
)
while kind_of_measurement in range(1, 4):
    print("\nInterpolación en proceso...\n")
    first_interpolator(kind_of_measurement, cleaned_dates)
    kind_of_measurement = False


Interpolación en proceso...

Interpolación espacial exitosa
(23, 33, 72)


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed